In [1]:
import pandas as pd
from tensorflow import keras


In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:,3:13]
Y = dataset.iloc[:,13]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [5]:
geography = pd.get_dummies(X['Geography'], drop_first = 0)
gender = pd.get_dummies(X['Gender'], drop_first = 0)

In [6]:
X = pd.concat([X,geography,gender],axis = 1)
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain,Female,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,True,False,False,True,False
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,False,False,True,True,False
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,True,False,False,True,False
3,699,France,Female,39,1,0.00,2,0,0,93826.63,True,False,False,True,False
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,False,False,True,True,False


In [7]:
X = X.drop(['Geography','Gender'],axis = 1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain,Female,Male
0,619,42,2,0.00,1,1,1,101348.88,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,False,False,True,True,False


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from keras import Sequential
from keras.layers import Dense, Dropout, Activation

In [14]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
def create_model(leyars, activation):
  model = Sequential()
  for i, nodes in enumerate(leyars):
    if i ==0:
      model.add(Dense(nodes, input_dim = X_train.shape[1]))
      model.add(Activation('relu'))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation('relu'))
      model.add(Dropout(0.3))
  model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'glorot_uniform'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [16]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [19]:
layers = [[20], [40, 20], [45, 30, 15]]
# activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, batch_size=[128, 256], epochs=[30])

# Perform the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train, Y_train)

ValueError: Invalid parameter layers for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(layers=[20])`
Check the list of available parameters with `estimator.get_params().keys()`